In [1]:
import os
wd = os.getcwd()
wd = os.path.dirname(os.path.dirname(wd))
import sys
sys.path.append(wd+"\\src\\")

In [24]:
from powderlayer import PowderBD, PowderOvLayer, Powder
from tube import Tube
import numpy as np

In [22]:
def get_layer0(powder_name, om_q, delta):
    d = 125e-3
    q = 5.2
    tube = Tube([0,d*50], [d,d])
    opts =  powder_layer_dict_sample = {
            'type': 'powder',
            'powder': powder_name,
            'omega': om_q * q * 1000,  # грамм
            'delta': delta, # г/cm^3
            'p_0': 30e6, # начальное давление
            't_ign': 0.0, # начало горения
            'u_0': 0,     #начальная скорость
        }
    calc_settings_sample = {
        'cell_dx': 0.005,
        'n_cells_min': 50,
        'n_cells_max': 300,
        'GasFluxCalculator_kwargs': {'flux_type':2},
        'GridStrecher_kwargs': {}
    }
    return PowderOvLayer.get_standart(tube, 0, opts, calc_settings_sample)

In [23]:
get_layer0('4\\7', 1, 700)

PowderOvLayer(n_cells=121); 
        {'p_max':5000000.0, 'tau_min': 4.006232259839776e-05, 
        'u_max': 0.0, 'cs_max': 124.87454021397153, 'dx_min': 0.005002764114378722, 
        'x_1': 0.0, 'x_2': 0.605334457839831,
        'V_1':  0.0, 'V_2':  0.0  }
        { 'powder': r'4\7', 'z_max': 0.0, 't_ign'=0.0 }

In [103]:
def get_border(layer, tau, m1, pf_1):
    p = layer.ps[-1]
    v = layer.Vs_borders[-1]
    tube = layer.tube
    s = np.pi * 0.125**2/4 * 1.04
    pa = 0
    if p > pf_1 or abs(v) > 1E-3:
        f1 = (p - pa) * s
        v1 = v + (f1 / m1)*tau
        return v1
    else:
        return v

def forw_full(powder_name, om_q, delta, courant_number = 0.4):
    lr: PowderOvLayer = get_layer0(powder_name, om_q, delta)
    layers = [lr]
    for i in range(10000):
        if lr.xs_borders[-1] >= 50*0.125:
            break
        if lr.Vs_borders[-1] > 1900:
            break
        if lr.time > 0.1:
            return None
        tau = lr.get_tau_min() *courant_number
        v1 = get_border(lr, tau, 5.2, 100)
        lr = lr.step_simple(tau, 0, v1)
        layers.append(lr)
    else:
        return None
    return layers
#     return lr
    
def forw_one(powder_name, om_q, delta, courant_number = 0.4):
    om = om_q * 5.2
    w0=om / delta
    lrs = forw_full(powder_name, om_q, delta, courant_number)
    if lrs is None:
        return False, w0, 100, 600, 0
    v = lrs[-1].Vs_borders[-1]
    
    suc = True
    if v < 1900:
        suc = False
        
    p_max = 0
    for lr in lrs:
        pm = np.max(lr.ps)
        if pm > p_max:
            p_max = pm
    if p_max > 400e6:
        suc = False
    
    return suc, w0, v, p_max/1e6, len(lrs)

In [108]:
9.75/582.644

0.01673406059274617

In [104]:
forw_one('14\\1 тр В\\А', 9.44/5.2, 671)

(True, 0.014068554396423249, 1900.4132474215285, 394.820796577644, 2461)

In [105]:
def get_opt_foo(powder_name):
    results = []
    def inner(x):
        om_q = x[0]
        delta= x[1]
        suc, w0, v, p_max, i = forw_one(powder_name, om_q, delta, courant_number = 0.4)
        results.append((x, suc, w0, v, p_max, i))
        shtraf = 0
        if v > 1900:
            shtraf += abs(1900-v)/1
        if p_max > 400:
            shtraf += abs(p_max-400)/1
        return w0 + shtraf
    return inner, results

In [100]:
good_powders = []
for name in bd.all_powders_dict:
    p = bd.all_powders_dict[name]
    if p['I_k'] and 0.7 < p['I_k'] < 1.3:
        good_powders.append(name)
good_powders

['5\\1 д-25',
 '7\\1 УГ',
 '8\\1 УГ',
 '9\\7 МН',
 '9\\7 БП',
 '11\\1 БП',
 '12\\1 Тр',
 '12\\1 тр БП',
 '12\\1 тр МН',
 '12\\7',
 '12\\7 В\\А',
 '14\\1 тр В\\А',
 '14\\7',
 '14\\7 В\\А',
 '15\\1 тр В\\А',
 '15\\7',
 '16\\1 тр',
 '16\\1 тр В\\А',
 '17\\7',
 '18\\1 тр',
 'НДТ-3 16\\1',
 'ДГ-3 13\\1',
 'ДГ-3 14\\1',
 'ДГ-4 13\\1',
 '100\\70',
 'ДГ-4 14\\1',
 'ДГ-4 15\\1',
 ' АПЦ-235 П 16\\1',
 'БНГ-1355 25\\1',
 'МАП-1 23\\1']

In [101]:
from scipy.optimize import differential_evolution

In [102]:
bounds = [(1,4), (500, 900)]

In [97]:
foo, results = get_opt_foo('16\\1 тр В\\А')
result = differential_evolution(foo, bounds)
result.x, result.fun

suc 455
suc 455
suc 455
suc 455
suc 455
suc 455
suc 455
suc 455
suc 455
suc 455
suc 455
suc 455
suc 455
suc 455
suc 455
suc 455
suc 455
suc 455
suc 455
suc 455
suc 455
suc 455
suc 455
suc 455


(array([  1.00000906, 899.97618119]), 0.005777983062030247)

In [98]:
forw_one('16\\1 тр В\\А', 1, 900)

(False, 0.005777777777777778, 1234.3416737912537, 202.79980062514167, 1541)